# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Nettoyer les données

In [4]:
#Création de la nouvelle dataframe(1) à partir du fichier csv
new_yt_df = df_youtube.copy()

#Changement de l'id
new_yt_df['video_id'] = pd.Series(range(1, (len(new_yt_df['video_id'])+1)))

#Mise en place de liste dans la colonne tags
tags_collumn = new_yt_df['tags']
for i in range (len(tags_collumn)):
    tags_collumn[i] = tags_collumn[i].split("|")
new_yt_df['tags'] = tags_collumn

#On associe le fichier json à une dataframe(2)
df_cat = pd.read_json("./data/US_category_id.json")
#On fait une dataframe(3) à partir de la série de dictionnaire (dernière colonne de notre dataframe(2) précédente) afin d'avoir les id
new_df_catid = pd.DataFrame(list(df_cat['items']))
#On fait une dataframe(4) à partir du dictionnaire présent dans le dictionnaire de la colonne 'snipper' de la dataframe(3) précedente afin d'avoir le titre des catégories
new_df_cat = pd.DataFrame(list(pd.DataFrame(list(df_cat['items']))['snippet']))
#On associe les deux dataframes((3)+(4)) différentes afin d'avoir les id que nous pouvons associer aux titres des catégories
new_df_id = pd.concat([new_df_catid, new_df_cat], axis=1)
#On convertie les id de ce dataframe((3)+(4)) en int afin d'avoir le même type que les id de la dataframe(1) du fichier csv
new_df_id["id"]=pd.to_numeric(new_df_id['id'], downcast='integer')
#On change le nom de la colonne des catégories afin d'avoir le même nom entre nos deux dataframes(1) et effectuer la jointure
new_df_id = new_df_id.rename(columns={'id':"category_id"})
#On effectue la jointure sur la colonne qui nous intéresse
new_df = pd.merge(new_yt_df, new_df_id, on="category_id")
#On rename quelques colonnes pour avoir une meilleure lisibilité
new_df = new_df.rename(columns={'title_x':"video_title", 'title_y':"category_title"})
#Nous supprimons les colonnes superflues provenant du fichier json
new_df = new_df.drop(['kind', 'etag', 'snippet', 'channelId'], axis=1)

<ipython-input-4-3d704cc6e2a6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tags_collumn[i] = tags_collumn[i].split("|")


### Importer les données

In [5]:
collection.delete_many({})
collection.insert_many(new_df.to_dict(orient='records'))

## Question 1  

In [6]:
#On récupère toutes lignes dont le 'channel_title' est 'Apple'.
cur = collection.find({'channel_title' : "Apple"}, {'video_id':1, 'video_title':1, 'channel_title':1})
list(cur)

[{'_id': ObjectId('5fce1bc8791f2aef9f8ae072'),
  'video_id': 2,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('5fce1bc8791f2aef9f8ae082'),
  'video_id': 204,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('5fce1bc8791f2aef9f8ae095'),
  'video_id': 420,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('5fce1bc8791f2aef9f8ae0aa'),
  'video_id': 675,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('5fce1bc8791f2aef9f8ae0bc'),
  'video_id': 906,
  'video_title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('5fce1bc8791f2aef9f8ae0ca'),
  'video_id': 1062,
  'video_title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple'},
 {'_id': ObjectId('5fce1bc8791f2aef9f8ae0de'),
  'video_id': 

## Question 2

In [7]:
#On compte le nombre d'éléments dans la liste contenant toutes les 'category_id' distinctes.
NbCat = len(collection.distinct('category_id'))
print("Nombre de catégories : "+str(NbCat))

Nombre de catégories : 16


## Question 3

In [8]:
#Nous avons déjà réalisé cette question dans le nettoyage des bases de donnée via pandas

## Question 4

In [9]:
#Retourne les 10 vidéos les plus vues.
cur = collection.find({}, {'video_id':1, 'video_title':1, 'views':1}).sort([('views', -1)])
list(cur)[0:10]


[{'_id': ObjectId('5fce1bc8791f2aef9f8aeef3'),
  'video_id': 1908,
  'video_title': "BTS (방탄소년단) 'DNA' Official MV",
  'views': 41500672},
 {'_id': ObjectId('5fce1bc8791f2aef9f8aeedf'),
  'video_id': 1695,
  'video_title': "BTS (방탄소년단) 'DNA' Official MV",
  'views': 38013692},
 {'_id': ObjectId('5fce1bc8791f2aef9f8aedca'),
  'video_id': 178,
  'video_title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'views': 36323498},
 {'_id': ObjectId('5fce1bc8791f2aef9f8adfbd'),
  'video_id': 7136,
  'video_title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'views': 33191594},
 {'_id': ObjectId('5fce1bc8791f2aef9f8adf91'),
  'video_id': 6918,
  'video_title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'views': 32512343},
 {'_id': ObjectId('5fce1bc8791f2aef9f8aeef9'),
  'video_id': 1996,
  'video_title': 'Shakira - Perro Fiel (Official Video) ft. Nicky Jam',
  'views': 32136948},
 {'_id': ObjectId('5fce1bc8791f2aef9f8adf63'),
  'video_id': 6673,
  'video_t

## Question 5

In [10]:
#Retourne le nombre de vue moyen par catégorie
cur = collection.aggregate([{'$group' : {'_id' : '$category_title', "moyenne de vue": {"$avg" : "$views"}}}])
list(cur)

[{'_id': 'Autos & Vehicles', 'moyenne de vue': 607693.8706896552},
 {'_id': 'Shows', 'moyenne de vue': 8492.5},
 {'_id': 'Travel & Events', 'moyenne de vue': 464041.0833333333},
 {'_id': 'Science & Technology', 'moyenne de vue': 924730.548828125},
 {'_id': 'Pets & Animals', 'moyenne de vue': 651404.3879310344},
 {'_id': 'News & Politics', 'moyenne de vue': 540955.5569823436},
 {'_id': 'Gaming', 'moyenne de vue': 681081.8170731707},
 {'_id': 'Comedy', 'moyenne de vue': 1240073.294039735},
 {'_id': 'Sports', 'moyenne de vue': 728434.3243902439},
 {'_id': 'Film & Animation', 'moyenne de vue': 1039472.6560846561},
 {'_id': 'Music', 'moyenne de vue': 1176553.612},
 {'_id': 'Entertainment', 'moyenne de vue': 1154868.4397251718},
 {'_id': 'Education', 'moyenne de vue': 547582.4700598803},
 {'_id': 'People & Blogs', 'moyenne de vue': 971532.7573696146},
 {'_id': 'Nonprofits & Activism', 'moyenne de vue': 1110334.2142857143},
 {'_id': 'Howto & Style', 'moyenne de vue': 537665.1806674339}]

## Question 6 

In [11]:
#Retourne les 10 chaines youtubes avec la plus grande moyenne de likes
cur = collection.aggregate([
    {'$group' : {'_id' : '$channel_title', 'moyenne de like': { "$avg" : "$likes"}}},
    {'$sort' : {'moyenne de like' : -1}}
])
list(cur)[0:10]


[{'_id': 'ZaynVEVO', 'moyenne de like': 1431683.0},
 {'_id': 'ibighit', 'moyenne de like': 1371766.25},
 {'_id': 'melanie martinez', 'moyenne de like': 911871.5},
 {'_id': 'BETNetworks', 'moyenne de like': 769687.625},
 {'_id': 'jypentertainment', 'moyenne de like': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'moyenne de like': 653443.5384615385},
 {'_id': 'Wengie', 'moyenne de like': 567235.2},
 {'_id': 'Alesso', 'moyenne de like': 509417.28571428574},
 {'_id': 'PewDiePie', 'moyenne de like': 508647.8},
 {'_id': 'shakiraVEVO', 'moyenne de like': 434302.14285714284}]